In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import time
from tqdm import tqdm as ProgressBar
import six # needed for Google Cloud client
import sys
import pickle
import operator
import nltk
import en # NodeBox https://www.nodebox.net/code/index.php/Linguistics#verb_conjugation

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:

df=pd.read_pickle('CleanedIngredientsWithVerbs.pkl')
ingredientsDict=pd.read_pickle('unique_ingredients.pkl')
ingredientsDict.append('oister')

In [3]:
df.shape
ingredientsDict.append('oyster')
'oyster' in ingredientsDict

True

In [4]:
ingredientsDict[0:9]

['',
 '(r) condensed cream celery soup',
 '(r) condensed cream mushroom soup',
 'abalone shell',
 'absinthe',
 'acai berry pulp',
 'acai berry sorbet',
 'acai powder',
 'acaipomegranate juice']

In [5]:
# df.sample(1)
print df[['instructionSteps']].values[0]
#print df.values[0]

[ [u'In a bowl, mix the tomatoes, onion, red bell pepper, yellow bell pepper, cilantro, lime juice, and salt. Cover and refrigerate until ready to serve.']]


In [6]:
print df[['ingredients']].values[0]

[ [u'6 roma (plum) tomatoes, diced', u'1 sweet onion, diced', u'1 medium red bell pepper, diced', u'1 medium yellow bell pepper, diced', u'1 bunch cilantro, finely minced', u'1 lime, juiced', u'1 teaspoon salt, or to taste', u'Add all ingredients to list', u'Add all ingredients to list']]


In [7]:
df["ingredients"].sample(5)

17776    [3 eggs, 1 1/2 cups milk, 1/2 cup butter, melt...
342      [3 cups water, 2 (14 ounce) cans diced tomatoe...
6588     [1 tablespoon vegetable oil, divided, 1 pinch ...
7438     [4 pounds fresh young black walnuts (in shell)...
7076     [1 teaspoon vegetable oil, 1 (13.8 ounce) pack...
Name: ingredients, dtype: object

In [8]:
df.sample(1)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes,cleanedIngredients,verbs
3704,"[Chicken Legs, Baked and Roasted Chicken, Main...",PT3H20M,NaN,"[8 large chicken legs, 1 tablespoon salt, 1 cu...",[Sprinkle chicken with salt and place in a lar...,Roasted Orange Rosemary Honey Glazed Chicken,4.32,53,http://allrecipes.com/recipe/235165/roasted-or...,200.0,"[chicken legs, salt, honey, orange juice pulp,...","[sprinkle, whisk, pour, cover, refrigerate, po..."


In [43]:
def get_ingredients(instructions, Ingredients, ingredientsDict):
    """Create a more generic list of ingredients using the way they are referenced in the instructions."""
    UnreferencesIngredients = list(Ingredients)
    client = language.LanguageServiceClient()
    joined_ingredients = ' '.join(UnreferencesIngredients)
    tokenized_ingredients = nltk.word_tokenize(joined_ingredients)
    
    if isinstance(instructions, six.binary_type):
        instructions = instructions.decode('utf-8')+'.'

    # Instantiates a plain text document.
    document = types.Document(
        content=instructions,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    short_ingredients=list()
    idx = 0
    content = list()
    for token in tokens:
        # Parser fails many times to identify an impertive verb in the beginning of a sentence
        # So if it can be a verb and is capitalized then it is a verb.
        tag = pos_tag[token.part_of_speech.tag]
        if token.text.content.lower() != token.text.content and en.is_verb(token.text.content.lower()):
            tag = 'VERB'
        # If this was identified and a noun and can be a verb and there is a noun following
        elif (content==[] # first
             and tag == 'NOUN' # parser thinks it is a NOUN but can also be a verb
             and len(tokens) > idx+1
             and en.is_verb(token.text.content.lower()) and pos_tag[tokens[idx+1].part_of_speech.tag] == 'NOUN'):
            if (pos_tag[tokens[idx-1].part_of_speech.tag] == 'PUNCT') and token.text.content not in ingredientsDict:
                tag = 'VERB'
        else:
            tag = pos_tag[token.part_of_speech.tag]

        # There is another noun coming, collect the whole phrase.
        # PRT example: confectioners' sugar
        # CONJ example: salt and pepper
        if (tag in ['NOUN','NUM','ADJ','PRT']) and len(tokens) > idx+1 and pos_tag[tokens[idx+1].part_of_speech.tag] in ['NOUN','PRT']:
            content.append(token.text.content)
            idx += 1
            continue
        else:
            content.append(token.text.content)
            # Combine the words to a phrase. Since apostrophe is a POS (PRT) I need to eliminate the extra space.
            term = ' '.join(content).replace(' \'','\'')
            if tag == 'NOUN' and term in UnreferencesIngredients: # term === Ingredient
                tag = 'INGREDIENT'
                UnreferencesIngredients.remove(term) 
                # print(u'{}: {}'.format(tag, term))
            elif (tag == 'NOUN' and (en.noun.singular(term) in tokenized_ingredients or en.noun.plural(term) in tokenized_ingredients)):
                tag = 'INGREDIENT'
                UnreferencesIngredients.remove(
                            [ingredient for ingredient in UnreferencesIngredients 
                             if term in ingredient or en.noun.singular(term) in ingredient or en.noun.plural(term) in ingredient][0]) 
            elif (' ' in term and tag == 'NOUN' and term in joined_ingredients):
                tag = 'INGREDIENT'
                ingSet = [ingredient for ingredient in UnreferencesIngredients if term in ingredient]
                if ingSet<>[]:
                    UnreferencesIngredients.remove(ingSet[0]) 
            else:
                if (' ' in term and tag == 'NOUN'):
                    if len(set([i for i in UnreferencesIngredients for word in term.split() if word in i])) == 1:
                        for i in set([i for i in UnreferencesIngredients for word in term.split() if word in i]):
                            UnreferencesIngredients.remove(i) 
                        tag = 'INGREDIENT'
                    if term == ' '.join([word for i in UnreferencesIngredients for word in term.split() if word in i]):
                        for i in set([i for i in UnreferencesIngredients for word in term.split() if word in i]):
                            UnreferencesIngredients.remove(i) 
                        tag = 'INGREDIENT'
                else:
                    # print(u'{}: {}'.format(tag, term))
                    pass
            content = list()
            idx += 1
        if tag == 'INGREDIENT':
            short_ingredients.append(en.noun.singular(term))
            joined_ingredients = ' '.join(UnreferencesIngredients)
            tokenized_ingredients = nltk.word_tokenize(joined_ingredients)
        # print(u'{}: {}'.format(tag, term))
    #time.sleep(.03)   # avoid Google API quota
    return(list(set(short_ingredients + UnreferencesIngredients)))

In [16]:
text="Whisk together the light soy sauce, cornstarch, 1 teaspoon of sugar, and pepper in a large glass or ceramic bowl. Add the chicken breast and toss to evenly coat. Cover the bowl with plastic wrap, and marinate in the refrigerator for 10 minutes..Heat the water in a large skillet over medium-high heat until boiling. Quickly stir in the chicken and cook for 3 to 4 minutes. Chicken will not be cooked through. Drain the water and place chicken on a plate. Set aside. Carefully wipe down the skillet and heat the vegetable oil over medium heat. Cook and stir the ginger and garlic until fragrant. Stir in the chicken, black beans and 1 tablespoon of sugar. Cook the chicken breasts until no longer pink in the center. Stir in the red and green peppers, green onions, carrots, oyster sauce, and thick soy sauce. Cook and stir for 1 minutes."
Ingredients=[u'1 pound ground beef', u'1/2 cup diced onion', u'2/3 cup chunky salsa', u'1/2 cup ketchup', u'1/2 cup packed brown sugar', u'1 egg', u'1/2 cup quick cooking oats', u'salt and pepper to taste', u'Add all ingredients to list', u'Add all ingredients to list']
cleanedIngredients =['light soy sauce', 'cornstarch', 'white sugar', 'black pepper', 'skinless boneless chicken breast meat', 'water', 'vegetable oil', 'ginger', 'garlic', 'fermented black beans', 'white sugar', 'green bell pepper', 'red bell pepper', 'green onions', 'carrot', 'oyster sauce', 'soy sauce']
client = language.LanguageServiceClient()

if isinstance(text, six.binary_type):
    text = text.decode('utf-8')

# Instantiates a plain text document.
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
#tokens = client.analyze_syntax(document).tokens
tokens = client.annotate_text(document, {'extract_syntax': True,}).tokens
pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
           'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')
idx = 0
content=list()
short_ingredients=list()
print len(tokens)
for token in tokens:
    if token.text.content.lower() != token.text.content and en.is_verb(token.text.content.lower()):
        tag = 'VERB'
    elif en.is_verb(token.text.content.lower()) and pos_tag[tokens[idx+1].part_of_speech.tag] == 'NOUN':
        tag = 'VERB'
    else:
        tag = pos_tag[token.part_of_speech.tag]
    
    # There is another noun coming, collect the whole phrase
    if (tag == 'NOUN' or tag == 'NUM' or tag == 'ADJ') and pos_tag[tokens[idx+1].part_of_speech.tag] == 'NOUN':
        content.append(token.text.content)
        idx += 1
        continue
    else:
        content.append(token.text.content)
        term = ' '.join(content)
        if tag == 'NOUN' and term in Ingredients:
            tag = 'INGREDIENT'
            Ingredients.remove(term)
            # print(u'{}: {}'.format(tag, term))
        elif (tag == 'NOUN' and term in ', '.join(Ingredients)):
            tag = 'INGREDIENT'
        else:
            if (' ' in term and tag == 'NOUN') or tag != 'NOUN':
                for word in term.split():
                    if word in Ingredients:
                        tag = 'INGREDIENT'
                        term = word
                        # print(u'{}: {}'.format(tag, term))
                    else:
                        # print(u'{}: {}'.format(tag, term))
                        pass
            else:
                # print(u'{}: {}'.format(tag, term))
                pass
        content = list()
        idx += 1
        if tag == 'INGREDIENT':
            short_ingredients.append(term)
print 'reduced: '+', '.join(sorted(set(short_ingredients)))
print 'original:'+', '.join(sorted(set(Ingredients)))

ingredients = get_ingredients(text, cleanedIngredients, ingredientsDict)
print ingredients
tonys = set(cleanedIngredients)
print 'We both have: ', (ingredients & tonys)          
print 'He doesn\'t have: ',ingredients-tonys
print 'I don\'t have: ',tonys-ingredients
print 

175
reduced: pepper, sugar
original:1 egg, 1 pound ground beef, 1/2 cup diced onion, 1/2 cup ketchup, 1/2 cup packed brown sugar, 1/2 cup quick cooking oats, 2/3 cup chunky salsa, Add all ingredients to list, salt and pepper to taste
set([u'vegetable oil', u'pepper', 'red bell pepper', u'light soy sauce', u'thick soy sauce', u'carrot', u'ginger', u'sugar', u'water', u'cornstarch', u'black bean', u'garlic', u'oyster sauce', 'green bell pepper', u'chicken breast', u'green onion'])
We both have:  set(['vegetable oil', 'red bell pepper', 'light soy sauce', 'ginger', 'water', 'cornstarch', 'carrot', 'garlic', 'green bell pepper', 'oyster sauce'])
He doesn't have:  set([u'pepper', u'thick soy sauce', u'sugar', u'black bean', u'chicken breast', u'green onion'])
I don't have:  set(['skinless boneless chicken breast meat', 'fermented black beans', 'soy sauce', 'black pepper', 'white sugar', 'green onions'])



In [11]:
ingredientsSet=set()
oldIngredientsSet=set()
idx=0
for idx, row in ProgressBar(df.iterrows(), desc="Processing recipes"):
    # print 'Ingredients:',row['cleanedIngredients']
    if type(row['instructionSteps']) != float:
        instructions = '.'.join(instruction for instruction in row['instructionSteps'])
        #print row['name']
        #print instructions
        #print row['ingredients']
        #print row['cleanedIngredients']
        ingredients = row['cleanedIngredients']
        oldIngredientsSet |=set(ingredients)
        # ingredients = ' '.join(ingredient for ingredient in row['cleanedIngredients'])
        ingredients = get_ingredients(instructions, ingredients, ingredientsDict)
        ingredientsSet |= ingredients
#    if idx==100:
#        break
print idx
print len(ingredientsSet)
print len(oldIngredientsSet)

Processing recipes: 89061it [10:58:28,  2.38it/s]

12678
17393
19013


In [9]:
print ingredientsSet
print oldIngredientsSet
with open('ingredientsSet.pkl', 'wb') as handle:
    pickle.dump(ingredientsSet, handle, protocol=pickle.HIGHEST_PROTOCOL)

set([])
set([])


In [ ]:
verbVertor =  [get_ingredients('.'.join(df[['instructionSteps']].values[i][0]),
                               df['cleanedIngredients'].values[i],
                               ingredientsDict
                              ) for i in range(df.shape[0])]
df['cleanerIngredients']=verbVertor

In [ ]:
# Store data (serialize to pickle file)
df.to_pickle('cleanerIngredients.pkl')